In [181]:
import geopandas as gpd

In [182]:
#get geodataframes from shapefile
gdf = gpd.read_file("Traffic_Collision_Data.zip")
speed_cameras = gpd.read_file("speed_cams.zip")

In [183]:
#remove unnessary columns
gdf = gdf.drop(
       columns=['ID', 'Accident_D', 'Accident_T', 'Classifica', 
              'Initial_Im', 'Road_Surfa', 'Environmen','Light', 'Traffic_Co', 'Num_of_Veh', 
              'Num_Of_Ped', 'Num_of_Bic','Num_of_Mot', 'Max_Injury', 'Num_of_Inj', 'Num_of_Min', 
              'Num_of_M_1','Num_of_Maj', 'Num_of_Fat'])
speed_cameras = speed_cameras.drop(
       columns=['January','February', 'March', 'April', 'May', 'June', 'July', 'August',
              'September', 'October', 'November', 'December'])

In [184]:
#Get only midblock collisions
road_gdf = gdf.loc[gdf['Location_T'] == "Midblock"]
#get all collisions on each road
all_counts = road_gdf['Location'].value_counts()
all_counts = all_counts.to_frame()

#get collisions for each year
years_gdf = all_counts
for year in range(2017,2023):
    year_gdf = gdf.loc[gdf['Accident_Y'] == str(year)] #get only collisions in year
    year_counts = year_gdf['Location'].value_counts() #count # of collisions
    year_counts = year_counts.rename("count_"+str(year)) #rename column
    years_gdf = years_gdf.merge(year_counts,how='left',on='Location') #merge to main gdf

#drop all duplicates
final_gdf = road_gdf.drop_duplicates(subset="Location")

#merge year count to points to get geometry
final_gdf = final_gdf.merge(years_gdf,how='left',on='Location').drop(columns=["Accident_Y"])

final_gdf

,Geo_ID,Location,Location_T,X_Coordina,Y_Coordina,Lat,Long,ObjectId,geometry,count,count_2017,count_2018,count_2019,count_2020,count_2021,count_2022
0,__5RG32R,WEST RIDGE DR btwn BERT G. ARGUE DR & PARLOR P...,Midblock,349455.3820,5012881.625,45.254477,-75.931063,1,POINT (-8452607.289 5661673.025),1,1.0,NaN,NaN,NaN,NaN,NaN
1,__3ZA2TN,VANIER PKWY SB btwn DONALD ST & VANIER PKWY (_...,Midblock,370277.0644,5032383.845,45.428323,-75.663225,2,POINT (-8422791.679 5689206.138),1,1.0,NaN,NaN,NaN,NaN,NaN
2,__3ZA705,EASTVALE DR btwn LOYOLA AVE & OGILVIE RD (__3Z...,Midblock,375627.4055,5035090.283,45.452153,-75.594468,7,POINT (-8415137.710 5692986.778),4,1.0,NaN,NaN,NaN,2.0,1.0
3,__3ZA198,WEST HUNT CLUB RD EB btwn CEDARVIEW RD & GREEN...,Midblock,361046.5881,5020239.248,45.319850,-75.782560,8,POINT (-8436076.039 5672016.688),14,4.0,4.0,4.0,1.0,NaN,1.0
4,__3ZA29K,BANK ST btwn COUNCILLOR'S WAY & ST. BERNARD ST...,Midblock,372911.0810,5023788.900,45.350738,-75.630753,12,POINT (-8419176.940 5676908.043),4,2.0,1.0,1.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8673,__3ZA91S,RICH LITTLE DR btwn BRIDLE CRT & COUNTRY CLUB ...,Midblock,368743.4398,5023203.707,45.345865,-75.684012,70793,POINT (-8425105.627 5676136.176),1,NaN,NaN,NaN,NaN,NaN,1.0
8674,__3ZA7IW,5TH LINE RD btwn DEVEREAUX RD & SNAKE ISLAND R...,Midblock,380976.8248,5006273.270,45.192311,-75.530518,70915,POINT (-8408018.781 5651847.945),1,NaN,NaN,NaN,NaN,NaN,1.0
8675,__3ZAC7P,BAYVIEW DR btwn LANE ST & STIVER RD (__3ZAC7P),Midblock,337653.9190,5040250.084,45.501399,-76.079595,70930,POINT (-8469141.801 5700804.901),1,NaN,NaN,NaN,NaN,NaN,1.0
8676,__3ZA0WI,ARTHUR ST btwn CHRISTIE ST & SOMERSET ST W (__...,Midblock,366841.7863,5030362.304,45.410447,-75.707377,70931,POINT (-8427706.648 5686371.091),1,NaN,NaN,NaN,NaN,NaN,1.0


In [185]:
#spatial join the counts to speed cameras lines
speed_cameras = speed_cameras.sjoin_nearest(final_gdf,how='left').fillna(0)

C:\Users\oppne\AppData\Local\Temp\ipykernel_35396\140711711.py:2: DeprecationWarning: ExtensionArray.fillna added a 'copy' keyword in pandas 2.1.0. In a future version, ExtensionArray subclasses will need to implement this keyword or an exception will be raised. In the interim, the keyword is ignored by GeometryArray.
  speed_cameras = speed_cameras.sjoin_nearest(final_gdf,how='left').fillna(0)


In [186]:
#save to file
speed_cameras.to_csv("outputs/year_counts.csv")